In [1]:
import numpy as np
import pandas as pd
import gurobipy as gp

from gurobipy import GRB

from pathlib import Path
import os
import sys
from datetime import datetime, date

In [2]:
result = Path('result/')
report = Path('report/')
instances = Path('instances/csifa')

In [3]:
file_name = sys.argv[1]

In [4]:
def read_instance(file_name):
	
	arq = open(file_name)
	
	N = int(arq.readline())
	
	PR  = np.zeros(N)
	PP  = np.zeros(N)

	FR = [float(arq.readline())]*N
	FP = [float(arq.readline())]*N

	HR = [float(arq.readline())]*N
	HP = [float(arq.readline())]*N

	D = [int(i) for i in arq.readline().split()]
	
	R = [int(i) for i in arq.readline().split()]

	C = float(arq.readline().rstrip('\n'))
	
	return N, PP, PR, FP, FR, HR, HP, D, R, C

In [5]:
MAX_CPU_TIME = 3600.0
EPSILON = 1e-6

def clsr_sp_mip(N, PP, PR, FP, FR, HR, HP, D, R, SD, SR, C):

	CSP = (np.zeros((N,N))).tolist()
	CSR = (np.zeros((N,N))).tolist()
	CR = (np.zeros((N,N))).tolist()
	CL = (np.zeros(N)).tolist()

	for i in range(N):
		for j in range(i,N):
			CR[i][j]  = sum(HR[t]*SR[i][t] for t in range(i,j))
			CSP[i][j] = PP[i] * SD[i][j] + sum(HP[t]*SD[t+1][j] for t in range(i,j))
			CSR[i][j] = PR[i] * SD[i][j] + sum(HP[t]*SD[t+1][j] for t in range(i,j))

	for i in range(N):
		CL[i] = sum(HR[j]*SR[i][j] for j in range(i,N))	

	try:

		# Create a new model
		model = gp.Model("clsr_sp_mip")

		# Create variables
		zsp = model.addVars([(i,j) for i in range(N) for j in range(i,N)],vtype=GRB.CONTINUOUS, name="zsp")
		zsr = model.addVars([(i,j) for i in range(N) for j in range(i,N)],vtype=GRB.CONTINUOUS, name="zsr")
		zr = model.addVars([(i,j) for i in range(N) for j in range(i,N)],vtype=GRB.CONTINUOUS, name="zr")
		l = model.addVars(list(range(N)), lb = 0.0, ub = 1.0, vtype=GRB.CONTINUOUS, name="l")
		yp = model.addVars(list(range(N)), lb = 0.0, ub = 1.0, vtype=GRB.BINARY, name="yp")
		yr = model.addVars(list(range(N)), lb = 0.0, ub = 1.0, vtype=GRB.BINARY, name="yr")

		# set objective
		
		#fobj = gp.quicksum(FP[i]*yp[i] for i in range(N))
		#fobj += gp.quicksum(FR[i]*yr[i] for i in range(N))
		#fobj += gp.quicksum(CL[i]*l[i] for i in range(N))

		#fobj = 0.0
		#for i in range(N):
		#	fobj += FP[i]*yp[i] + FR[i]*yr[i] + CL[i]*l[i]
        #   fobj += gp.quicksum(CSP[i][j]*zsp[i,j] + CSR[i][j]*zsr[i,j] + CR[i][j]*zr[i,j] for j in range(i,N))
		
		fobj = 0.0
		for i in range(N):
			fobj += FP[i]*yp[i] + FR[i]*yr[i] + CL[i]*l[i] 
			
			for j in range(i,N):
				fobj += CSP[i][j]*zsp[i,j] + CSR[i][j]*zsr[i,j] + CR[i][j]*zr[i,j] 

		model.setObjective(fobj, sense = GRB.MINIMIZE)

		# add constraints

		model.addConstr(
			gp.quicksum(zsp[0,j] + zsr[0,j] for j in range(N)) == 1
			)
		
		model.addConstrs(
			gp.quicksum(zsp[i,t-1] + zsr[i,t-1] for i in range(t)) - 
			gp.quicksum(zsp[t,j] + zsr[t,j] for j in range(t, N)) == 0  for t in range(1,N) 
			)
				
		model.addConstrs(
			gp.quicksum(zsp[t,j] for j in range(t,N)) <= yp[t] for t in range(N)
			)
			
		model.addConstrs(
			gp.quicksum(zsr[t,j] for j in range(t,N)) <= yr[t] for t in range(N)
			)
			
		model.addConstr(
			gp.quicksum(zr[0,j] for j in range(N)) + l[0]==1
			)
					
		model.addConstrs(
			gp.quicksum(zr[i,t-1] for i in range(0,t)) == 
			gp.quicksum(zr[t,j]  for j in  range(t,N)) + l[t] for t in range(1,N)
			)       
				
		model.addConstrs(
			gp.quicksum(zr[i,t] for i in range(0,t+1)) <= yr[t] for t in range(N)
			)    
			
		model.addConstrs(
			gp.quicksum(SR[i][t]*zr[i,t] for i in range(t+1) ) ==
			gp.quicksum(SD[t][j]*zsr[t,j] for j in range(t,N)) for t in range(N)
			)

#		model.addConstrs(
#			gp.quicksum(SD[i][t]*zsp[i,t] for t in range(i,N)) <= yp[i]*min(C,SD[i][N-1]) for i in range(N)
#			)
		model.addConstrs(
			gp.quicksum(SD[i][t]*zsp[i,t] for t in range(i,N)) <= yp[i]*SD[i][N-1] for i in range(N)
			)
		
		model.addConstrs(
			gp.quicksum(SD[i][t]*zsr[i,t] for t in range(i,N)) <= yr[i]*SD[i][N-1] for i in range(N)
			)
		
		model.addConstrs(
			gp.quicksum(SD[t][k]*zsp[t,k] for k in range(t,N)) + 
			gp.quicksum(SD[t][k]*zsr[t,k] for k in range(t,N)) <= C for t in range(N)
			)
		
		# Parameters 
		model.setParam(GRB.Param.TimeLimit, 600)
		model.setParam(GRB.Param.MIPGap, 1.e-6)
		model.setParam(GRB.Param.Threads, 1)
		model.setParam(GRB.Param.Method, 1)
		model.setParam(GRB.Param.NodeMethod, 1)
		#model.setParam(GRB.Param.Cuts, -1)
		#model.setParam(GRB.Param.Presolve, -1)

		# turn off display
		gp.setParam('OutputFlag', 0)

		# open log file
		_ = open('log/clspr_sp.log', 'w')

		# Optimize model
		model.optimize()

		tmp = 0
		if model.status == GRB.OPTIMAL:
			tmp = 1

		#print('Obj: %g' % model.ObjVal)

	except gp.GurobiError as e:
		print('Error code ' + str(e.errno) + ': ' + str(e))
		
		
	objval = model.ObjVal
	objbound = model.ObjBound
	mipgap = model.MIPGap
	runtime = model.Runtime
	nodecount = model.NodeCount

	return objval, objbound, mipgap, runtime, nodecount, tmp


In [6]:
def main(file_name):
	
	N, PP, PR, FP, FR, HR, HP, D, R, C = read_instance(os.path.join(instances,file_name))

	SD = (np.zeros((N,N))).tolist()
	SR = (np.zeros((N,N))).tolist()

	for  i in range(N):
		SD[i][i] = D[i]
		SR[i][i] = R[i]
		for j in range(i+1,N):
			SD[i][j] = SD[i][j-1] + D[j]
			SR[i][j] = SR[i][j-1] + R[j]

	bestsol, bestbound, gap, runtime, numnode, status = clsr_sp_mip(N, PP, PR, FP, FR, HR, HP, D, R, SD, SR, C)
	
	arquivo = open(os.path.join(result,'clsr_sp_mip.csv'),'a')
	
	arquivo.write(file_name+';'
			   +str(round(bestsol,2))+';'
			   +str(round(bestbound,2))+';'
			   +str(round(gap,2))+';'
			   +str(round(runtime,2))+';'
			   +str(round(numnode,3))+';'
			   +str(round(status,2))+'\n')

	arquivo.close()

In [7]:
if __name__== "__main__" :

	for dim in [52]:
		for id in range(1,11):
			file_name = f"c{dim}_{id}.txt"			
			main(file_name)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-02-19
Set parameter TimeLimit to value 3600
Set parameter MIPGap to value 1e-06
Set parameter Threads to value 1
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 1 threads

Optimize a model with 468 rows, 4290 columns and 20826 nonzeros
Model fingerprint: 0x92898b21
Variable types: 4186 continuous, 104 integer (104 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [4e+00, 1e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective 18092.600000
Presolve removed 103 rows and 1 columns
Presolve time: 0.02s
Presolved: 365 rows, 4289 columns, 18016 nonzeros
Variable types: 4186 continuous, 103 integer (103 binary)

Root relaxation: objective 8.605767e+03, 701 iter